#CMSC421 Final Project
Team Members: Jordan Maggin, Lex Kim, Zhuo Cheng Xie, Neel Jay, Hylene Wu

Project Description: The program will train on the set of images and gather information on airplanes from these images. With this information, it will attempt to both locate and identify airplanes in different sets of videos.


Installing YOLO

In [1]:
!nvidia-smi

Mon May  6 17:30:23 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 546.65                 Driver Version: 546.65       CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1050 Ti   WDDM  | 00000000:01:00.0  On |                  N/A |
| 35%   29C    P8              N/A /  75W |    963MiB /  4096MiB |      3%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip install ultralytics==8.0.196

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.196  Python-3.11.7 torch-2.3.0+cpu CPU (Intel Core(TM) i7-6700 3.40GHz)
Setup complete  (8 CPUs, 31.9 GB RAM, 1284.7/1862.4 GB disk)


In [3]:
from ultralytics import YOLO
from IPython.display import display, Image


Importing dataset from Roboflow Project

In [3]:
import os
root_dir = os.getcwd()
!mkdir {root_dir}\datasets
%cd {root_dir}\datasets

# paste the exported dataset code here
!pip install roboflow --quiet

from roboflow import Roboflow
rf = Roboflow(api_key="nr1RFEWRZXe3o3eQV1ns")
project = rf.workspace("cmsc421-final-project").project("vehicle-detection-ansgq")
version = project.version(1)
dataset = version.download("yolov8")

print("Success!")

C:\Users\wizar.GW\Downloads\final421\datasets


A subdirectory or file C:\Users\wizar.GW\Downloads\final421\datasets already exists.


loading Roboflow workspace...
loading Roboflow project...
Success!


Custom Training

(Weights are saved to /content/runs/detect/train)

In [4]:
%cd {root_dir}

!yolo task=detect mode=train model=yolov8s.pt data={dataset.location}/data.yaml epochs=25 imgsz=640 plots=True

C:\Users\wizar.GW\Downloads\final421
^C


Resulting Confusion Matrix

In [5]:
Image(filename=f'{root_dir}/runs/detect/train/confusion_matrix.png', width=600)
# SAVE IMAGEPATH IN VARIABLE
confusion = 

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\wizar.GW\\Downloads\\final421/runs/detect/train/confusion_matrix.png'

In [ ]:
Image(filename=f'{root_dir}/runs/detect/train/results.png', width=600)
# SAVE IMAGEPATH AS VARIABLE
results = 

Validate model

In [ ]:
!yolo task=detect mode=val model={root_dir}/runs/detect/train/weights/best.pt data={dataset.location}/data.yaml

Inference/Predict with Model

In [ ]:
!yolo task=detect mode=predict model={root_dir}/runs/detect/train/weights/best.pt conf=0.25 source={dataset.location}/test/images save=True

In [ ]:
Image(filename=f'{root_dir}/runs/detect/train/val_batch2_pred.jpg', width=600)
# SAVE IMAGEPATH AS VARIABLE
predictions = f'{root_dir}/runs/detect/train/val_batch2_pred.jpg'

Deploying to Roboflow

In [ ]:
project.version(dataset.version).deploy(model_type="yolov8", model_path=f"{root_dir}/runs/detect/train/")

# Simple GUI with PySimpleGUI

In [8]:
# Installing PySimpleGUI
!python -m pip install pysimplegui
!pip install pillow

In [15]:
# Imports
import PySimpleGUI as sg
import cv2
import numpy as np
from PIL import Image, ImageTk
import io
import os

In [24]:
# Defines potential filetypes to search for
file_types = [("JPEG (*.jpg)", "*.jpg"),
              ("All files (*.*)", "*.*")]

def main():
    # Defines list of files from selected folder
    file_list_column = [
        [
            sg.Text("Image Folder"),
            sg.In(size=(25, 1), enable_events=True, key="-FOLDER-"),
            sg.FolderBrowse(),
        ],
        [
            sg.Listbox(
                values=[], enable_events=True, size=(40, 20), key="-FILE LIST-"
            )
        ],
    ]
    
    # File loader
    file_loader_column = [
        [
            sg.Text("Image File"),
            sg.Input(size=(25, 1), key="-FILE-"),
            sg.FileBrowse(file_types=file_types),
        ],
        [
            sg.Button("Load Image"),
        ]
    ]
    
    # Image display
    image_display_column = [
        [
            sg.Image(key="-IMAGE-"),
        ]
    ]
    
    # Defines the layout of the program
    layout = [
        #[sg.Image(key="-IMAGE-")],
        [
            sg.Column(file_list_column, element_justification='c'),
#             sg.Column(file_loader_column, element_justification='c'),
            sg.VSeparator(),
            sg.Column(image_display_column, element_justification='c'),
        ],
        [
            sg.Button("Confusion Matrix"),
            sg.Button("Result Graphs"),
            sg.Button("Predictions"),
        ],
    ]
    
    
    # Applies the layout to the window
    window = sg.Window("Image Viewer", layout, element_justification='c')
    
    ###########
    # UPDATER #
    ###########
    # Updates the window with corresponding values and images
    while True:
        event, values = window.read()
        if event == "Exit" or event == sg.WIN_CLOSED:
            break
        if event == "-FOLDER-":
            folder = values["-FOLDER-"]
            try:
                # Get list of files in folder
                file_list = os.listdir(folder)
            except:
                file_list = []

            fnames = [
                f
                for f in file_list
                if os.path.isfile(os.path.join(folder, f))
                and f.lower().endswith((".jpg"))
            ]
            window["-FILE LIST-"].update(fnames)
        if event == "-FILE LIST-":  # A file was chosen from the listbox
            try:
                filename = os.path.join(
                    values["-FOLDER-"], values["-FILE LIST-"][0]
                )
                image = Image.open(filename)
                image.thumbnail((400, 400))
                bio = io.BytesIO()
                image.save(bio, format="PNG")
                window["-IMAGE-"].update(data=bio.getvalue())
            except:
                pass
        if event == "Load Image":
            filename = values["-FILE-"]
            if os.path.exists(filename):
                # NOTE: If you want to do predictions, they would have to be placed here, prior to conversion.
                image = Image.open(values["-FILE-"])
                image.thumbnail((400, 400))
                bio = io.BytesIO()
                image.save(bio, format="PNG")
                window["-IMAGE-"].update(data=bio.getvalue())
        if event == "Confusion Matrix":
            window["-IMAGE-"].update(confusion)
        if event == "Result Graphs":
            window["-IMAGE-"].update(results)
        if event == "Predictions":
            window["-IMAGE-"].update(predictions)
    
    # Closes window as necessary
    window.close()

main()